## Dependencies

In [1]:
using LinearAlgebra
using ForwardDiff
using Plots

## Algorithms

In [2]:
# Gradient Descent
function gradient_descent(f, X, y, β_0, k_max, ϵ_tol)
    k = 1; β = β_0;
    J = ForwardDiff.jacobian(β -> f(X, β), β)
    err = (y - f(X, β)) # m * 1
    h = J' * err # n * 1
    
    # Store data
    F_array = [err' * err]
    h_array = [norm(h, Inf)]
    β_array = β
    
    while (k < k_max) & (norm(h, Inf) > ϵ_tol)
        α = 0.01 
        β = vec(β + α * h) # n * 1
        J = ForwardDiff.jacobian(β -> f(X, β), β)
        err = (y - f(X, β))
        h = J' * err # n * m x m * 1 = n * 1
        
        # update
        F_array = vcat(F_array, err' * err)
        h_array = vcat(h_array, norm(h, Inf))
        β_array = hcat(β_array, β)
        k += 1
    end
    return β, k, norm(h, Inf), F_array, h_array, β_array
end

gradient_descent (generic function with 1 method)

In [3]:
# Gauss-Newton
function gauss_newton(f, X, y, β_0, k_max, ϵ_tol)
    k = 1; β = β_0; h = Inf
    J = ForwardDiff.jacobian(β -> f(X, β), β)
    err = (y - f(X, β)) # m * 1
    g = J' * err # n * 1
    
    # Store data
    F_array = [err' * err]
    g_array = [norm(g, Inf)]
    β_array = β
    
    while (k < k_max) & (norm(h, Inf) > ϵ_tol) & (norm(g, Inf) > ϵ_tol)        
        h = (J' * J) \ g # n * n x n * 1 = n * m x m * 1
        β = vec(β + h) # n * 1
        J = ForwardDiff.jacobian(β -> f(X, β), β)
        err = (y - f(X, β)) # m * 1
        g = J' * err # n * 1
        
        # update
        F_array = vcat(F_array, err' * err)
        g_array = vcat(g_array, norm(g, Inf))
        β_array = hcat(β_array, β)
        k += 1
    end
    return β, k, norm(h, Inf), norm(g, Inf), F_array, g_array, β_array
end

gauss_newton (generic function with 1 method)

In [4]:
# LM
function lm_og(f, X, y, β_0, k_max, ϵ_tol, τ_0)
    k = 1; β = β_0; h = Inf; τ = τ_0
    J = ForwardDiff.jacobian(β -> f(X, β), β)
    Jev = 1
    A = J' * J # n * n 
    λ = τ * maximum(Diagonal(J' * J))
    err = (y - f(X, β)) # m * 1
    g = J' * err # n * 1
    F = (err' * err)
    
    # Store data
    F_array = [F]
    g_array = [norm(g, Inf)]
    λ_array = [λ]
    β_array = β
    
    while (k < k_max) & (norm(h, Inf) > ϵ_tol) & (norm(g, Inf) > ϵ_tol)
        h = (A + λ * I) \ g # n * n x n * 1 = n * m x m * 1 = n * 1 
        β_new = vec(β + h) # n * 1
        
        # construct rho
        err_h = (y - f(X, β_new)) # m * 1
        F_h = (err_h' * err_h) # float
        L = (h' * (λ * h + g)) # 1 * n x n * 1 = 1 * 1 
        ρ = (F - F_h) / L # float
        
        if ρ > 0
            β = β_new
            J = ForwardDiff.jacobian(β -> f(X, β), β)
            Jev += 1
            A = J' * J # n * n 
            err = (y - f(X, β)) # m * 1
            g = J' * err # n * 1
            F = F_h
        end
        if ρ > 0.8
            λ = λ / 3
            # λ = max(λ / 9, 1e-7)
        end
        if ρ < 0.2
            λ = λ * 2    
            # λ = min(λ * 11, 1e7)
        end
        
        # update
        F_array = vcat(F_array, err' * err)
        g_array = vcat(g_array, norm(g, Inf))
        λ_array = vcat(λ_array, λ)
        β_array = hcat(β_array, β)
        k += 1
    end
    return β, k, norm(h, Inf), norm(g, Inf), Jev, F_array, g_array, λ_array, β_array
end

lm_og (generic function with 1 method)

In [5]:
# LM (N)
function lm_nielsen(f, X, y, β_0, k_max, ϵ_tol, τ_0, ν)
    k = 1; β = β_0; h = Inf; τ = τ_0
    J = ForwardDiff.jacobian(β -> f(X, β), β)
    Jev = 1
    A = J' * J # n * n 
    λ = τ * maximum(Diagonal(J' * J))
    err = (y - f(X, β)) # m * 1
    g = J' * err # n * 1
    F = (err' * err)
    
    # Store data
    F_array = [F]
    g_array = [norm(g, Inf)]
    λ_array = [λ]
    β_array = β
    
    while (k < k_max) & (norm(h, Inf) > ϵ_tol) & (norm(g, Inf) > ϵ_tol)
        h = (A + λ * I) \ g # n * n x n * 1 = n * m x m * 1 = n * 1 
        β_new = vec(β + h) # n * 1
        
        # construct rho
        err_h = (y - f(X, β_new)) # m * 1
        F_h = (err_h' * err_h) # float
        L = (h' * (λ * h + g)) # 1 * n x n * 1 = 1 * 1 
        
        ρ = (F - F_h) / L # float
        if ρ > 0
            β = β_new
            J = ForwardDiff.jacobian(β -> f(X, β), β)
            Jev += 1
            A = J' * J # n * n 
            err = (y - f(X, β)) # m * 1
            g = J' * err # n * 1
            F = F_h
            λ = λ * max(1/3, 1 - (2*ρ - 1)^3)
            # λ = max(λ / 9, 1e-7)
            ν = 2
        else
            # λ = min(λ * 11, 1e7)
            λ = λ * ν
            ν = 2 * ν
        end
        
        # update
        F_array = vcat(F_array, err' * err)
        g_array = vcat(g_array, norm(g, Inf))
        λ_array = vcat(λ_array, λ)
        β_array = hcat(β_array, β)
        k += 1
    end
    return β, k, norm(h, Inf), norm(g, Inf), Jev, F_array, g_array, λ_array, β_array
end

lm_nielsen (generic function with 1 method)

## Experiments

#### 1. Linear function, full rank

In [6]:
n = 4; m = 100;
A = vcat(I(n) - 2*ones((n, n))/m, -2*ones((m-n,n))/m);
b = -ones(n);

lin_fr(A, b) = A * b - ones(m);
y = A * b - ones(m);
b_0 = ones(n);
τ_0 = 1e-8;

In [7]:
@time gradient_descent(lin_fr, A, y, b_0, 10000, 1e-12);

  2.962983 seconds (10.25 M allocations: 782.661 MiB, 7.95% gc time, 94.02% compilation time)


In [8]:
@time gauss_newton(lin_fr, A, y, b_0, 10000, 1e-12);

  2.817575 seconds (10.82 M allocations: 564.300 MiB, 5.29% gc time, 99.92% compilation time)


In [9]:
@time lm_og(lin_fr, A, y, b_0, 10000, 1e-12, τ_0);

  1.549266 seconds (6.07 M allocations: 326.495 MiB, 15.66% gc time, 99.85% compilation time)


In [10]:
@time lm_nielsen(lin_fr, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.366383 seconds (5.93 M allocations: 319.131 MiB, 6.81% gc time, 99.87% compilation time)


#### 2. Linear function, rank 1

In [11]:
n = 4; m = 100;
A = [i for i=1:m] * [i for i=1:n]';
b = 3*ones(n)/(sum(1:n) * (2*m + 1));

lin_r1(A, b) = A * b - ones(m);
y = A * b - ones(m);
b_0 = ones(n);
τ_0 = 1e-8;

In [12]:
@time gradient_descent(lin_r1, A, y, b_0, 10000, 1e-12);

  1.277073 seconds (5.71 M allocations: 302.845 MiB, 7.19% gc time, 99.82% compilation time)


In [13]:
@time gauss_newton(lin_r1, A, y, b_0, 10000, 1e-12);

LoadError: SingularException(2)

In [14]:
@time lm_og(lin_r1, A, y, b_0, 10000, 1e-12, τ_0);

  1.281428 seconds (5.96 M allocations: 320.104 MiB, 6.75% gc time, 99.87% compilation time)


In [15]:
@time lm_nielsen(lin_r1, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.339572 seconds (5.96 M allocations: 320.297 MiB, 6.81% gc time, 99.87% compilation time)


#### 3. Rosenbrock function

In [16]:
n = 2; m = 2;
A = [10, 1];
b = ones(n);

rosenbrock(A, b) = [A[1]*(b[2] - b[1]^2), A[2] - b[1]];
y = zeros(m);
b_0 = [-1.2, 1];
τ_0 = 1;

In [17]:
@time gradient_descent(rosenbrock, A, y, b_0, 10000, 1e-12);

  2.109711 seconds (6.07 M allocations: 1.809 GiB, 11.34% gc time, 61.54% compilation time)


In [18]:
@time gauss_newton(rosenbrock, A, y, b_0, 10000, 1e-12);

  1.078819 seconds (5.32 M allocations: 289.932 MiB, 6.62% gc time, 99.91% compilation time)


In [19]:
@time lm_og(rosenbrock, A, y, b_0, 10000, 1e-12, 1);

  1.114676 seconds (5.35 M allocations: 291.867 MiB, 6.56% gc time, 99.88% compilation time)


In [20]:
@time lm_nielsen(rosenbrock, A, y, b_0, 10000, 1e-12, 1, 2);

  1.124759 seconds (5.35 M allocations: 291.682 MiB, 7.16% gc time, 99.87% compilation time)


#### 4. Powell singular function

In [21]:
n = 4; m = 4;
A = [10, sqrt(5), 2, sqrt(10)];
b = zeros(n);

powell(A, b) = [b[1] + A[1] * b[2], A[2] * (b[3] - b[4]), (b[2] - A[3] * b[3])^2, A[4] * (b[1] - b[4])];
y = zeros(m);
b_0 = [3, -1, 0, 1];
τ_0 = 1e-8;

In [22]:
@time gradient_descent(powell, A, y, b_0, 10000, 1e-12);

  2.308749 seconds (6.35 M allocations: 2.572 GiB, 11.10% gc time, 59.10% compilation time)


In [23]:
@time gauss_newton(powell, A, y, b_0, 10000, 1e-12);

  1.163380 seconds (5.38 M allocations: 292.778 MiB, 5.92% gc time, 99.90% compilation time)


In [24]:
@time lm_og(powell, A, y, b_0, 10000, 1e-12, τ_0);

  1.176728 seconds (5.40 M allocations: 294.717 MiB, 4.95% gc time, 99.89% compilation time)


In [25]:
@time lm_nielsen(powell, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.290872 seconds (5.41 M allocations: 294.703 MiB, 7.38% gc time, 99.88% compilation time)


#### 5. Freudenstein and Roth function

In [26]:
n = 2; m = 2;
A = [2, 5, 13, 14, 1, 29];
b = [11.4128, -0.896805];

fr(A, b) = [b[1] - b[2] * (A[1] - b[2] * (A[2] - b[2])) - A[3], b[1] - b[2] * (A[4] - b[2] * (A[5] + b[2])) - A[6]];
y = zeros(m);
b_0 = [0.5, -2];
τ_0 = 1;

In [27]:
output = @time gradient_descent(fr, A, y, b_0, 10000, 1e-12);

  1.224681 seconds (5.27 M allocations: 282.463 MiB, 6.50% gc time, 99.90% compilation time)


In [28]:
output_gn = @time gauss_newton(fr, A, y, b_0, 10000, 1e-12);

  1.158789 seconds (5.33 M allocations: 290.386 MiB, 7.72% gc time, 99.88% compilation time)


In [29]:
output_og = @time lm_og(fr, A, y, b_0, 10000, 1e-12, τ_0);

  1.263207 seconds (5.36 M allocations: 292.238 MiB, 6.96% gc time, 99.88% compilation time)


In [30]:
output_n = @time lm_nielsen(fr, A, y, b_0, 1000, 1e-12, τ_0, 2);

  1.249051 seconds (5.36 M allocations: 292.074 MiB, 6.63% gc time, 99.87% compilation time)


In [31]:
xs = range(-40, stop=60, length=100)
ys = range(-5, stop=9, length=100)
f(x,y) = log((x - y * (A[1] - y * (A[2] - y)) - A[3])^2 + (x - y * (A[4] - y * (A[5] + y)) - A[6])^2)
zs = [f(x,y) for y in ys, x in xs]
Plots.contour(xs, ys, zs, c = :acton)
plot!(output_gn[7][1, :], output_gn[7][2, :], label = "gn", linecolor = "red")
plot!(output_n[9][1, :], output_n[9][2, :], label = "LM_n", linecolor = "blue")
plot!(output_og[9][1, :], output_og[9][2, :], label = "LM", linecolor = "green")
Plots.savefig("contour.png")

#### 6. Bard function

In [32]:
n = 3; m = 15;
u = [i for i=1:m];
v = [m + 1 - i for i=1:m];
w = [min(u[i], v[i]) for i=1:m];
A = hcat(u,v,w);
b = [0.082411, 1.133036, 2.343695];

bard(A, b) = b[1] .+ A[:, 1] ./ (b[2] .* A[:, 2] + b[3] .* A[:, 3]);
y = [0.14, 0.18, 0.22, 0.25, 0.29, 0.32, 0.35, 0.39, 0.37, 0.58, 0.73, 0.96, 1.34, 2.10, 4.39];
b_0 = ones(n);
τ_0 = 1e-8;

In [33]:
@time gradient_descent(bard, A, y, b_0, 10000, 1e-12);

  2.814852 seconds (7.11 M allocations: 2.262 GiB, 10.90% gc time, 62.88% compilation time)


In [34]:
@time gauss_newton(bard, A, y, b_0, 10000, 1e-12);

  1.608806 seconds (6.34 M allocations: 338.885 MiB, 5.53% gc time, 99.92% compilation time)


In [35]:
@time lm_og(bard, A, y, b_0, 10000, 1e-12, τ_0);

  1.619609 seconds (6.36 M allocations: 340.438 MiB, 5.67% gc time, 99.92% compilation time)


In [36]:
@time lm_nielsen(bard, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.644711 seconds (6.37 M allocations: 340.672 MiB, 6.90% gc time, 99.92% compilation time)


#### 7. Box 3-dimensional function

In [37]:
n = 3; m = 100;
A = [i/10 for i=1:m];
b = [1, 10, 1];

box3d(A, b) = exp.(-b[1] .* A) - exp.(-b[2] .* A) - b[3] .* (exp.(-A) - exp.(-10 .* A));
y = zeros(m);
b_0 = [0, 10, 20];
τ_0 = 1e-8;

In [38]:
@time gradient_descent(box3d, A, y, b_0, 10000, 1e-12);

  2.614786 seconds (7.61 M allocations: 2.539 GiB, 10.29% gc time, 66.89% compilation time)


In [39]:
@time gauss_newton(box3d, A, y, b_0, 10000, 1e-12);

  1.561341 seconds (6.43 M allocations: 344.391 MiB, 5.93% gc time, 99.92% compilation time)


In [40]:
@time lm_og(box3d, A, y, b_0, 10000, 1e-12, τ_0);

  1.621585 seconds (6.46 M allocations: 346.540 MiB, 6.50% gc time, 99.91% compilation time)


In [41]:
@time lm_nielsen(box3d, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.650004 seconds (6.47 M allocations: 346.660 MiB, 6.55% gc time, 99.90% compilation time)


#### 8. Jennrich and Sampson function

m = 5

In [42]:
n = 2; m = 5;
A = [i for i=1:m];
b = [0.378468, 0.378468];

js(A, b) = 2 .+ 2 .* A - (exp.(b[1] .* A) + exp.(b[2] .* A));
y = zeros(m);
b_0 = [0.3, 0.4];
τ_0 = 1;

In [43]:
@time gradient_descent(js, A, y, b_0, 10000, 1e-12);

  2.117393 seconds (7.01 M allocations: 368.920 MiB, 5.86% gc time, 99.93% compilation time)


In [44]:
@time gauss_newton(js, A, y, b_0, 10000, 1e-12);

  1.937215 seconds (6.60 M allocations: 352.344 MiB, 6.37% gc time, 99.92% compilation time)


In [45]:
@time lm_og(js, A, y, b_0, 10000, 1e-12, τ_0);

  1.786072 seconds (6.63 M allocations: 354.171 MiB, 6.03% gc time, 99.92% compilation time)


In [46]:
@time lm_nielsen(js, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.745220 seconds (6.63 M allocations: 354.061 MiB, 4.98% gc time, 99.92% compilation time)


m = 10

In [47]:
n = 2; m = 10;
A = [i for i=1:m];
b = [0.257825, 0.257825];

js(A, b) = 2 .+ 2 .* A - (exp.(b[1] .* A) + exp.(b[2] .* A));
y = zeros(m);
b_0 = [0.3, 0.4];
τ_0 = 1;

In [48]:
@time gradient_descent(js, A, y, b_0, 10000, 1e-12);

  0.319503 seconds (914.72 k allocations: 51.768 MiB, 6.64% gc time, 99.95% compilation time)


In [49]:
@time gauss_newton(js, A, y, b_0, 10000, 1e-12);

  0.339657 seconds (972.23 k allocations: 60.386 MiB, 6.00% gc time, 99.93% compilation time)


In [50]:
@time lm_og(js, A, y, b_0, 10000, 1e-12, τ_0);

  0.357669 seconds (1.00 M allocations: 62.170 MiB, 5.26% gc time, 99.90% compilation time)


In [51]:
@time lm_nielsen(js, A, y, b_0, 10000, 1e-12, τ_0, 2);

  0.349212 seconds (1.00 M allocations: 62.117 MiB, 3.80% gc time, 99.89% compilation time)


m = 20

In [52]:
n = 2; m = 20;
A = [i for i=1:m];
b = [0.165191, 0.165191];

js(A, b) = 2 .+ 2 .* A - (exp.(b[1] .* A) + exp.(b[2] .* A));
y = zeros(m);
b_0 = [0.3, 0.4];
τ_0 = 1;

In [53]:
@time gradient_descent(js, A, y, b_0, 10000, 1e-12);

  0.309627 seconds (914.70 k allocations: 51.770 MiB, 99.95% compilation time)


In [54]:
@time gauss_newton(js, A, y, b_0, 10000, 1e-12);

  0.343976 seconds (972.01 k allocations: 60.373 MiB, 3.84% gc time, 99.94% compilation time)


In [55]:
@time lm_og(js, A, y, b_0, 10000, 1e-12, τ_0);

  0.361539 seconds (999.99 k allocations: 62.192 MiB, 3.79% gc time, 99.90% compilation time)


In [56]:
@time lm_nielsen(js, A, y, b_0, 10000, 1e-12, τ_0, 2);

  0.350415 seconds (1.00 M allocations: 62.203 MiB, 3.73% gc time, 99.88% compilation time)


#### 9. Osborne 1 function

In [57]:
n = 5; m = 33;
A = [10 * (i - 1) for i=1:m];
b = [0.37541, 1.93585, -1.46469, 0.01287, 0.02212];

osborne(A, b) = b[1] .+ b[2] .* exp.(-b[4] .* A) + b[3] .* exp.(-b[5] .* A);
y = [0.844, 0.908, 0.932, 0.936, 0.925, 0.908, 0.881, 0.850, 0.818, 0.784, 0.751, 0.718, 0.685, 0.658, 0.628, 0.603, 0.580, 0.558, 0.538, 0.522, 0.506, 0.490, 0.478, 0.467, 0.457, 0.448, 0.438, 0.431, 0.424, 0.420, 0.414, 0.411, 0.406];
b_0 = [0.5, 1.5, -1, 0.01, 0.02];
τ_0 = 1e-8;

In [58]:
@time gradient_descent(osborne, A, y, b_0, 10000, 1e-12);

  2.855519 seconds (7.27 M allocations: 3.056 GiB, 11.26% gc time, 62.92% compilation time)


In [59]:
@time gauss_newton(osborne, A, y, b_0, 10000, 1e-12);

  1.700250 seconds (6.64 M allocations: 352.596 MiB, 4.58% gc time, 99.91% compilation time)


In [60]:
@time lm_og(osborne, A, y, b_0, 10000, 1e-12, τ_0);

  1.732004 seconds (6.66 M allocations: 354.217 MiB, 6.74% gc time, 99.91% compilation time)


In [61]:
@time lm_nielsen(osborne, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.776092 seconds (6.67 M allocations: 358.453 MiB, 5.93% gc time, 99.91% compilation time)


#### 10. Exponential fit, 4 parameters

In [62]:
n = 4; m = 45;
A = [0.02 * i for i=1:m];
b = [-4, -5, 4, -4];

exp4(A, b) = b[3] .* exp.(-b[1] .* A) + b[4] .* exp.(-b[2] .* A);
y = b[3] .* exp.(-b[1] .* A) + b[4] .* exp.(-b[2] .* A)
b_0 = [-1, -2, 1, -1];
τ_0 = 1e-3;

In [63]:
@time gradient_descent(exp4, A, y, b_0, 10000, 1e-12);

  1.617788 seconds (6.22 M allocations: 327.387 MiB, 6.57% gc time, 99.92% compilation time)


In [64]:
@time gauss_newton(exp4, A, y, b_0, 10000, 1e-12);

  1.596154 seconds (6.19 M allocations: 331.271 MiB, 5.04% gc time, 99.92% compilation time)


In [65]:
output_og = @time lm_og(exp4, A, y, b_0, 10000, 1e-12, τ_0);

  1.851839 seconds (6.66 M allocations: 360.343 MiB, 6.56% gc time, 99.64% compilation time)


In [69]:
plot([i for i=1:output_og[2]], hcat(output_og[6], output_og[7], output_og[8]), yaxis=:log, label = ["F" "||g||" "λ"], seriestype = :scatter, title = "Original", fmt = :png)
Plots.savefig("original.png")

In [67]:
output_n = @time lm_nielsen(exp4, A, y, b_0, 10000, 1e-12, τ_0, 2);

  1.669776 seconds (6.23 M allocations: 336.255 MiB, 6.22% gc time, 99.79% compilation time)


In [68]:
plot([i for i=1:output_n[2]], hcat(output_n[6], output_n[7], output_n[8]), yaxis=:log, label = ["F" "||g||" "λ"], seriestype = :scatter, title = "Nielsen", fmt = :png)
Plots.savefig("nielsen.png")